In [1]:
import torch.nn as nn
import torch
import gc

# Paquete de optimizadores
import torch.optim as optim

from sklearn.metrics import accuracy_score

In [2]:
def train_unet(model, train_dataloader, optm, loss_criterion, device):
    
    # Lista para las perdidas de cada época
    train_loss = []
    
    # Para calcular la precisión
    correct = 0
    total = 0
    
    # Modo de entrenamiento del modelo
    model.train()
    
    # Para cada imagen y mascara del conjunto
    for images, masks in train_dataloader:
        
        # Permuta las dimensiones de las imágenes a (N, C, H, W)
        images = images.permute(0, 3, 1, 2)
        
        images, masks = images.to(device, dtype=torch.float), masks.to(device,  dtype=torch.long)

        # Realiza la inferencia
        outputs = model(images)
        
        # Calcula la pérdida
        loss = loss_criterion(outputs, masks)
        
        # Limpia los gradientes del optimizador
        optm.zero_grad()
        
        # Realiza la retropropagación y actualiza los pesos del modelo
        loss.backward()
        optm.step()
  
        # Calcula la precisión, predicted es un tensor que contiene la clase predicha con la mayor probabilidad para cada píxel en la salida del modelo
        _, predicted = torch.max(outputs.data, 1)
        total += masks.nelement()  # Número total de píxeles en la máscara
        correct += predicted.eq(masks.data).sum().item()  # Número total de píxeles correctamente clasificados
        
        # Liberamos memoria
        del images, masks
        gc.collect()
        
        train_loss.append(loss.item())

    accuracy = round((correct / total)*100,2)  # Precisión del modelo
    
    return train_loss, accuracy